Un importante broker de compra y venta de vehiculos online se encuentra dando sus
primeros pasos en la preparación de su algoritmo de pricing, es por eso que se encuentra
generando algunos features iniciales para experimentar con distintos algoritmos de machine
learning.
Para ello cuenta con un archivo con información de todas las transacciones que tuvo en su
primer año de operación en el formato (transaction_id, timestamp, vehicle_model_id, price).
Por otro lado cuenta con información que fue extrayendo a partir de scrapping durante el último
año en el formato (timestamp, source, vehicle_model_id, price). La información puede venir de
múltiples fuentes (source), que pueden ser por ejemplo distintos sitios de marketplace o
clasificados.
Luego de un intenso trabajo previo ha podido unificar los modelos de vehículos que utiliza para
sus transacciones con la información que ha podido obtener de otros competidores mediante
scrapping. Muchos de los modelos disponibles en la información de scrapping no han sido aún
comercializados por la empresa, pero se sabe que se cuenta con precios scrapeados de todos
los modelos que se vendieron.
Se pide generar utilizando Pandas un dataframe que tenga el siguiente formato
(vehicle_model_id, ext_mean_price, ext_std_price, int_mean_price, int_std_price), siendo:
- mean_price: precio promedio para ese vehículo.
- std_price: desvío estándar del precio para ese vehículo.
y los prefijo ext_ y int_ indicando que deben ser calculado sobre respectivamente datos
externos (los obtenido vía scraping) y datos internos (los de las transacciones).

In [186]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random

transacciones = pd.DataFrame(columns = ['id', 'timestamp','auto_id', 'precio'], dtype = int)
id = [2, 9, 28, 6, 1, 25]
timestamp = ['2020-12-06 05:59:01','2019-09-06 05:55:01', '2020-04-20 04:20:42', '2019-12-06 05:45:01']
anio = [2017, 2017, 2016, 1999, 1999, 2015, 2015]
auto_id = [20, 200, 280, 10, 10, 250, 250]
precio = [200, 2000, 2800, 100, 100, 2500, 25]
source = ['a', 'b', 'c', 'd']
info = pd.DataFrame(columns = ['timestamp', 'source', 'auto_id', 'precio'], dtype = int)
for i in range(800):
    transacciones = transacciones.append({
        'id': random.choice(id),
        'timestamp': random.choice(timestamp),
        'auto_id': random.choice(auto_id),
        'precio': random.choice(precio)
    }, ignore_index = True)
    if i % 3 == 0:
        info = info.append({
            'timestamp': random.choice(timestamp),
            'source': random.choice(source),
            'auto_id': random.choice(auto_id),
            'precio': random.choice(precio)
        }, ignore_index = True)


In [187]:
interno = transacciones[['auto_id', 'precio']]
externo = info[['auto_id', 'precio']]
externo.dtypes

auto_id    int64
precio     int64
dtype: object

In [188]:
interno = interno.groupby('auto_id').agg({'precio':['mean', 'std']}).reset_index()
level0 = interno.columns.get_level_values(0)
level1 = interno.columns.get_level_values(1)
interno.columns = level0 + '_' + level1
interno


,auto_id_,precio_mean,precio_std
0,10,1102.531646,1191.479249
1,20,904.521277,1147.747694
2,200,1085.049020,1153.348868
3,250,1204.198473,1202.281327
4,280,1067.380952,1176.911926


In [189]:
externo = externo.groupby('auto_id').agg({'precio':['mean', 'std']}).reset_index()
level00 = externo.columns.get_level_values(0)
level11 = externo.columns.get_level_values(1)
externo.columns = level00 + '_' + level11
externo

,auto_id_,precio_mean,precio_std
0,10,1232.246377,1219.247458
1,20,1134.000000,1230.766191
2,200,1342.129630,1239.263553
3,250,950.000000,1103.089855
4,280,1105.714286,1187.822490


In [190]:
final = externo.merge(interno, on = 'auto_id_', how = 'inner')
final

,auto_id_,precio_mean_x,precio_std_x,precio_mean_y,precio_std_y
0,10,1232.246377,1219.247458,1102.531646,1191.479249
1,20,1134.000000,1230.766191,904.521277,1147.747694
2,200,1342.129630,1239.263553,1085.049020,1153.348868
3,250,950.000000,1103.089855,1204.198473,1202.281327
4,280,1105.714286,1187.822490,1067.380952,1176.911926


y hay que renombrar